## Dota Game Assistant

This script retrieves and summarizes information about a specified hero from `dotabuff.com` website

In [ ]:
!pip install selenium
!pip install webdriver-manager

In [ ]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [ ]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

In [ ]:
openai = OpenAI()

In [ ]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

class Website:
    def __init__(self, url, wait_time=10):
        """
        Create this Website object from the given URL using Selenium and BeautifulSoup.
        Uses headless Chrome to load JavaScript content.
        """
        self.url = url

        # Configure headless Chrome
        options = Options()
        options.headless = True
        options.add_argument("--disable-gpu")
        options.add_argument("--no-sandbox")

        # Start the driver
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)

        try:
            driver.get(url)

            # Wait until body is loaded (you can tweak the wait condition)
            WebDriverWait(driver, wait_time).until(
                EC.presence_of_element_located((By.TAG_NAME, "body"))
            )

            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")

            self.title = soup.title.string.strip() if soup.title else "No title found"

            # Remove unwanted tags
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()

            self.text = soup.body.get_text(separator="\n", strip=True)

        finally:
            driver.quit()


In [ ]:
class DotaWebsite:
    def __init__(self, hero):
        web = Website("https://www.dotabuff.com/heroes" + "/" + hero)
        self.title = web.title
        self.text = web.text

In [ ]:
system_prompt = "You are an game assistant that analyzes the contents of a website \
and provides a short summary about facet selection, ability building, item building, best versus and worst versus, ignoring text that might be navigation related. \
Respond in markdown."

In [ ]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of provides a short summary about facet selection, ability building, item building, best versus and worst versus in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [ ]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [ ]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(hero):
    website = DotaWebsite(hero)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [ ]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(hero):
    summary = summarize(hero)
    display(Markdown(summary))

In [ ]:
display_summary("axe")